In [1]:
import os, re
import tensorflow as tf
import keras.backend as K
import SimpleITK as sitk
import numpy as np
import keras
import cv2
import time
from scipy import ndimage
from tqdm import tqdm as tqdm
from skimage.measure import label,regionprops
from scipy.ndimage import measurements
from skimage.filters import roberts
from numpy import convolve
from skimage import measure
from skimage.segmentation import clear_border
from scipy import ndimage as ndi
from statsmodels.tsa.api import ExponentialSmoothing

#### Extraction Helper Functions

In [2]:
#kreating kernel for morphological transformations
def get_kernel():
    kernel=np.zeros((11,11),np.uint8)
    kernel=cv2.circle(kernel,(5,5),4,1,-1)
    for i in [4,6]:
        for j in [1,9]:
            kernel[i,j]=1
            kernel[j,i]=1
    return kernel

def find_bounds_z_direction(image):
    z_dist = np.sum(image,axis = (1,2))     
    zi = np.where(z_dist!=0)
    min_bound = np.min(zi)
    max_bound = np.max(zi)
    center_pos = int((max_bound - min_bound)/2)
    return center_pos,min_bound,max_bound

def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values, weights, 'valid')
    return sma

def resize_3d_img(img,shape,interp = cv2.INTER_NEAREST):
    init_img = img.copy()
    temp_img = np.zeros((init_img.shape[0],shape[1],shape[2]))
    new_img = np.zeros(shape)
    for i in range(0,init_img.shape[0]):
        temp_img[i,...] = cv2.resize(init_img[i,...],dsize=(shape[2],shape[1]),interpolation = interp)   
    for j in range(0,temp_img.shape[1]):
        new_img[:,j,:] = (cv2.resize(temp_img[:,j,:],dsize=(shape[2],shape[0]),interpolation = interp)) 
    return new_img

def create_flipped_mask(image,i_orig,kernel,method=0):   #image is a lung mask image, i_orig is an image volume
    _,min_bound,max_bound = find_bounds_z_direction(image)
    fl_mask = np.zeros_like(image)
    lc_buff=[]
    alpha = 0.3
    lungs_center_ind ={}
    
    if method:
    ## With adjustment for spine senter irregularity
        st_pt=[]
        lc_buff =[]
        for i in range(min_bound,min_bound+5):
            rib_view = 1*(i_orig[i,...] > 100)
            rib_view[370:,:] = 0
            distr = np.sum(rib_view,axis=0)
            filtered_distr = movingaverage(distr, 7)
            c_limit = int(0.15 * rib_view.shape[0])
            thresh_l1 = np.max(distr)
            distr_indicies = np.where(distr > (thresh_l1*0.15))
            lungs_center = int(rib_view.shape[0]/2-c_limit)+np.where(filtered_distr[int(rib_view.shape[0]/2-c_limit):int(rib_view.shape[0]/2+c_limit)] == np.max(filtered_distr[int(rib_view.shape[0]/2-c_limit):int(rib_view.shape[0]/2+c_limit)]))[0][0]
            st_pt.append(lungs_center)
            
        s0 = np.mean(st_pt)  
        for i in range(min_bound,max_bound+1):
            rib_view = 1*(i_orig[i,...] > 100)
            rib_view[370:,:] = 0
            distr = np.sum(rib_view,axis=0)
            filtered_distr = movingaverage(distr, 7)
            c_limit = int(0.15 * rib_view.shape[0])
            thresh_l1 = np.max(distr)
            distr_indicies = np.where(distr > (thresh_l1*0.15))
            lungs_center = int(rib_view.shape[0]/2-c_limit)+np.where(filtered_distr[int(rib_view.shape[0]/2-c_limit):int(rib_view.shape[0]/2+c_limit)] == np.max(filtered_distr[int(rib_view.shape[0]/2-c_limit):int(rib_view.shape[0]/2+c_limit)]))[0][0]
            st_pt.append(lungs_center)
            print(lungs_center)
            lungs_center_ind[i] = int((alpha*lungs_center)+(1-alpha)*s0)
            s0 = lungs_center
        
    else:
    ## quick method no adjustment
        for i in range(min_bound,max_bound+1):
            rib_view = 1*(i_orig[i,...]>100)
            distr = np.sum(rib_view,axis=0)
            thresh_l1 = np.max(distr)
            distr_indicies = np.where(distr > (thresh_l1*0.15))
            lc_buff.append((np.min(distr_indicies)+np.max(distr_indicies))/2)
        lungs_center = int(np.mean(lc_buff))
        for i in range(min_bound,max_bound+1):
            lungs_center_ind[i] = lungs_center    
        
        
    for i in range(min_bound,max_bound+1):
        ax = image[i,...]
        if lungs_center_ind[i] > int(image.shape[2]/2):
            part1 = ax[:,2*lungs_center_ind[i] -image.shape[2]:lungs_center_ind[i] ]
            part2 = np.hstack((ax[:,lungs_center_ind[i]:],np.zeros((image.shape[1],2*lungs_center_ind[i] -image.shape[2]))))
            part2 = cv2.dilate(np.array(part2,np.uint8),kernel,iterations=1)
            part1 = cv2.dilate(np.array(part1,np.uint8),kernel,iterations=1)
            fl_mask[i,...] = np.hstack((np.flip(part2,axis=1),np.flip(part1,axis=1)))
            un = np.array(fl_mask[i,...],np.uint8)|np.array(ax,np.uint8)
            fl_mask[i,...] = un
        elif lungs_center_ind[i]  < int(image.shape[2]/2):
            part1 = np.hstack((np.zeros((image.shape[1],image.shape[2]-2*lungs_center_ind[i] )),ax[:,:lungs_center_ind[i] ]))
            part2 = ax[:,lungs_center_ind[i] :image.shape[2]-(image.shape[2]-2*lungs_center_ind[i] )]
            part2 = cv2.dilate(np.array(part2,np.uint8),kernel,iterations=1)
            part1 = cv2.dilate(np.array(part1,np.uint8),kernel,iterations=1)
            fl_mask[i,...] = np.hstack((np.flip(part2,axis=1),np.flip(part1,axis=1)))
            un = np.array(fl_mask[i,...],np.uint8)|np.array(ax,np.uint8)
            fl_mask[i,...] = un
        else:
            fl = np.flip(ax,axis=1)
            fl_mask[i,...] = np.array(ax,np.uint8)|np.array(fl,np.uint8)
        
    return fl_mask,min_bound,max_bound
    

def largest_labeled_volumes(im, bg=-1,ratio=0.6):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        try:
            max_val = np.max(counts)
            counts = counts/max_val
            smax = np.where(counts == (counts[((counts>ratio)*(counts<1))]))[0][0]

            return vals[[np.argmax(counts),smax]]
        except:   
            return [vals[np.argmax(counts)]]
    else:
        return None
    
def max_connected_volume_extraction(image):
    #image should be int
    img_buff = image.copy()
    img_buff_lb = image.copy()
    img_buff = 1*(img_buff>0)
    img_buff_lb = np.array(img_buff_lb*10,np.uint8) 
    output_mask = np.zeros_like(img_buff)
    connectivity_array = np.zeros_like(img_buff)
    label_image = label(img_buff_lb)

    for i in range(0,len(img_buff)):
        if i == 0:
            connectivity_array[i,...] = img_buff[i+1,...]&img_buff[i,...]
        elif i == len(img_buff)-1:
            connectivity_array[i,...] = img_buff[i,...]&img_buff[i-1,...]
        else:
            connectivity_array[i,...] = (img_buff[i-1,...]&img_buff[i,...])|(img_buff[i+1,...]&img_buff[i,...]) 

    multip_arr = label_image*connectivity_array
    areas = list(np.unique(multip_arr[multip_arr>0]))
    ##volumetric post processing
    if True:
        if len(areas)>1:
            max_lab_ar = 1
            temp_max = np.sum(1*multip_arr==1)
            for a in areas:
                if np.sum(1*multip_arr==a)>temp_max:
                    temp_max = np.sum(1*multip_arr==a)
                    max_lab_ar =a
            for a in areas:
                #if np.sum(1*multip_arr==a)*params['original_spacing'][2]*0.001>1.5:
                #    pass
                #else:
                if a != max_lab_ar:
                    multip_arr[multip_arr==a]=0
                else:
                    pass
            areas = list(np.unique(multip_arr[multip_arr>0]))

    for val in areas:
        output_mask[label_image==val]=1
    return output_mask

    
def get_seg_lungs(img,return_only_mask= True):
    im = img.copy()
    kernel = get_kernel()
    # Convert into a binary image. 
    binary = (im < -320) 
    
    sq = img.shape[1]*img.shape[2]
    seg_lung =np.zeros_like(img)
    
    
    for nm,ax_slice in enumerate(binary):
    
        # Remove the blobs connected to the border of the image
        cleared = clear_border(ax_slice)
        # Label the image
        label_image = label(cleared)
        
        # Keep the labels with 3 largest areas
        areas = [r.area for r in regionprops(label_image)]
        areas.sort()
        if len(areas) > 3:
            for region in regionprops(label_image):
                if region.area < areas[-3]:
                    for coordinates in region.coords:                
                           label_image[coordinates[0], coordinates[1]] = 0
        binary_sl = label_image > 0
        
        
        # Fill in the small holes inside the lungs
        edges = roberts(binary_sl)
        binary_sl = ndi.binary_fill_holes(edges)
        seg_lung[nm,...] = binary_sl
        
    
    for i,axial_slice in enumerate(seg_lung):   #Delete Trahea from mask 
        trahea_coeff = 0.0053*(sq/(512**2))
        trahea_coeff = 0.0069*(sq/(512**2)) #sebast
        labels = measure.label(axial_slice,connectivity=1,background=0)

        vals, counts = np.unique(labels, return_counts=True)
        counts = counts[vals != 0]
        vals = vals[vals != 0]
        if (counts/(sq*1) < trahea_coeff).any():
            for l in vals[counts/(sq*1) < trahea_coeff]:
                labels[labels == l] = 0
            labels = labels != 0
            seg_lung[i,...] = labels  
                
    # Remove table and other air pockets insided body
    labels = measure.label(seg_lung, background=0)

    for i in np.unique(labels):
        temp_center = measurements.center_of_mass(labels==i)[1]
        if temp_center> seg_lung.shape[1]*0.75:
            seg_lung[labels==i]=0

    labels = measure.label(seg_lung, background=0)        
    l_max = largest_labeled_volumes(labels, bg=0)
    if l_max is not None: # There are air pockets
        if len(l_max)>1:
            labels[labels == l_max[1]] = l_max[0]
            seg_lung[labels != l_max[0]] = 0

        else:
             seg_lung[labels != l_max[0]] = 0
            
    
    # Searching for lungs center and flip the lungs
    seg_lung,min_bound,max_bound = create_flipped_mask(seg_lung,im,kernel)
           

    for i,axial_slice in enumerate(seg_lung):
        bi=np.array((axial_slice),np.uint8)
        opening = cv2.morphologyEx((1-bi),cv2.MORPH_OPEN,kernel,iterations=7)
        dilat = cv2.dilate((1-opening),kernel,iterations=2)
        seg_lung[i,...] = cv2.erode(dilat,kernel,iterations=1)
        
    #Stretch the lung mask in z direction +-2 slices
    if (min_bound>2)and (max_bound<seg_lung.shape[0]-2):
        try:

            for i in range(1,3):
                seg_lung[min_bound -i,...] = seg_lung[min_bound,...]
                seg_lung[max_bound +i,...] = seg_lung[max_bound,...]
            min_bound-=2
            max_bound+=2
        except:
            print('some problrm with patient ')
    if return_only_mask:
        return seg_lung
    else:
        return seg_lung,min_bound,max_bound
   
    
def apply_mask(image,mask,threshold=-1000): #apply mask to image and save HU values
    new_img = image.copy()
    new_img[np.where(mask==0)] = threshold
    
    return new_img

def parse_dataset(general_path,img_only = True):
    patients = next(os.walk(general_path))[1]
    Patient_dict={}
    temp_img=''
    temp_mask=''
    i=0
    
    for patient in tqdm(patients):
        for root, dirs, files in os.walk(os.path.join(general_path,patient)):
            for file in files:
                if not img_only:
                    if re.search('mask',file.lower()):
                        temp_mask = file
                        for imgfile in files:
                            if re.search('volume',imgfile.lower()) or re.search('CT',imgfile.lower()):
                                temp_img = imgfile
                                Patient_dict[i]=[os.path.join(general_path,patient,temp_img),os.path.join(general_path,patient,temp_mask)]
                                i+=1
                                temp_mask=''
                                temp_img=''
                else:
                    if re.search(r'_CT\.nrrd$', file, re.IGNORECASE):
                        temp_img = file
                        Patient_dict[i]=[os.path.join(general_path,patient,temp_img)]
                        i+=1
                        temp_img=''
                
    print(len(Patient_dict),r' Patients found')
    return Patient_dict
                

### Patient Data

In [3]:
class Patient_data_generator(keras.utils.Sequence):
    
    def __init__(self,patient_dict, batch_size=8, image_size=512,tumor_size_low_lim = 0,sl_th = None, zero_sl_ratio = 1, model_params={}, 
                 shuffle=False, augment=False, predict=False,img_only = False, normalize = False, norm_coeff=[], 
                 use_window = False, window_params=[],hu_intensity = False, resample_int_val = False, resampling_step =25,
                 equalize_hist = False, verbosity = False, size_eval = False, calculate_snr =False,reshape=False, extract_lungs=False):
        
        self.patient_dict = patient_dict
        self.filenames = list(patient_dict.keys())
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.hu_intensity = hu_intensity
        self.predict = predict
        self.normalize = normalize
        self.norm_coeff =norm_coeff
        self.equalize_hist=equalize_hist
        self.calculate_snr = calculate_snr
        self.use_window = use_window
        self.window_params = window_params
        self._tum_size = tumor_size_low_lim
        self._slice_thickness = sl_th
        self.zero_sl_ratio = zero_sl_ratio
        self.resample_int_val = resample_int_val
        self.step = resampling_step
        self.img_only = img_only
        self.tum_size_distr=[]
        self.tum_volume_distr={}
        self.zero_dict={}
        self.tum_dict={}
        self.reshape = reshape
        self.verbosity=verbosity
        self.extract_lungs = extract_lungs
        self.model_params=model_params
        self.on_epoch_end()
        self.__epoch_zero_dict = None
        self.__epoch_tum_dict = None
        self._normalization_init()
        self._patients_init(size_eval)
        
    def _resize_3d_img(self,img,shape,interp = cv2.INTER_CUBIC):
        init_img = img.copy()
        temp_img = np.zeros((init_img.shape[0],shape[1],shape[2]))
        new_img = np.zeros(shape)
        for i in range(0,init_img.shape[0]):
            temp_img[i,...] = cv2.resize(init_img[i,...],dsize=(shape[2],shape[1]),interpolation = interp)   
        for j in range(0,shape[1]):
            new_img[:,j,:] = (cv2.resize(temp_img[:,j,:],dsize=(shape[2],shape[0]),interpolation = interp)) 
        return new_img     
        
    def _load_patient(self, filename):
        # load patient files as numpy arrays 
        
        if self.verbosity:
            print('-'*7,'Loading patient ',filename,'-'*7)
        img = sitk.GetArrayFromImage(sitk.ReadImage(self.patient_dict[filename][0]))
        msk = sitk.GetArrayFromImage(sitk.ReadImage(self.patient_dict[filename][1]))
        if self.verbosity:
            print('-'*7,'Img spacing ',sitk.ReadImage(self.patient_dict[filename][0]).GetSpacing(),'-'*7)
            print('Image shape:',img.shape)
        
        #print(img.shape,msk.shape)
        #Set minimal values for HU intensities
        if self.hu_intensity:#Set lower limit to -1000 HU
            img[img <-1000] = -1000
            if self.verbosity:
                print('Adjust intensity') 
                
        if self.extract_lungs:
            temp_img = img.copy()
            temp_args = get_seg_lungs(img=temp_img,return_only_mask=False)
            if temp_args is not None:
                lung_mask,minb,maxb=temp_args
                lung_mask_with_tumor_mask = lung_mask.copy()
                kernel = get_kernel()
                
                
                ##stretch lung mask if the tumour is outside it, only for training
                kernel_sm =cv2.circle(kernel,(3,3),4,1,-1)
                lung_mask_indicies = np.where(np.sum(lung_mask,axis=(1,2))!=0)
                min_ind_l_mask,max_ind_l_mask = np.min(lung_mask_indicies),np.max(lung_mask_indicies)
                try:
                    tum_mask_indicies = np.where(np.sum(msk,axis=(1,2))!=0)
                    min_ind_tum_mask,max_ind_tum_mask = np.min(tum_mask_indicies),np.max(tum_mask_indicies)
                except:
                    min_ind_tum_mask,max_ind_tum_mask = min_ind_l_mask,max_ind_l_mask
                    
                    
                
                if min_ind_tum_mask < min_ind_l_mask:
                    temp_slice = lung_mask[min_ind_l_mask,...]
                    for i in range(min_ind_tum_mask,min_ind_l_mask):
                        tmp = temp_slice*(1*(img[i,...]<-400))
                        tmp = cv2.morphologyEx(np.array(tmp,np.uint8),cv2.MORPH_CLOSE,kernel_sm,iterations=4)
                        tmp = cv2.morphologyEx(np.array(tmp,np.uint8),cv2.MORPH_OPEN,kernel_sm,iterations=2)
                        lung_mask[i,...] = tmp
                    if self.verbosity:
                        print('Mask stretched down')
                    min_ind_l_mask = min_ind_tum_mask
                    
                if max_ind_tum_mask>max_ind_l_mask:
                    temp_slice = lung_mask[max_ind_l_mask,...]
                    for i in range(max_ind_l_mask,max_ind_tum_mask):
                        tmp = temp_slice*(1*(img[i,...]<-400))
                        tmp = cv2.morphologyEx(np.array(tmp,np.uint8),cv2.MORPH_CLOSE,kernel_sm,iterations=4)
                        tmp = cv2.morphologyEx(np.array(tmp,np.uint8),cv2.MORPH_OPEN,kernel_sm,iterations=2)
                        lung_mask[i,...] = tmp
                    if self.verbosity:
                        print('Mask stretched up')
                    max_ind_l_mask = max_ind_tum_mask
                    
                
                for k,layer in enumerate(msk):
                    temp_layer = layer.copy()
                    lung_mask_with_tumor_mask[k,...]= np.array(lung_mask_with_tumor_mask[k,...],np.uint8)|cv2.dilate(np.array(temp_layer,np.uint8),kernel,iterations=3)
        
                new_img = apply_mask(img,lung_mask_with_tumor_mask)
                img = new_img
                temp_list = set(self.__epoch_zero_dict[filename])
                for item in temp_list:
                    if (item<= min_ind_l_mask) or (item >= max_ind_l_mask):
                        self.__epoch_zero_dict[filename].remove(item)
                
                
                if self.verbosity:
                    print('Applying mask done ')
            else:
                print('Lungs are not extracted for patient',filename[0][:-11].split('\\')[-1])
                pass    
            
        ####  Reshaping the image&mask
        if True:
            img_for_reshape = img.copy()
            msk_for_reshape = msk.copy()
            temp_data_orig = sitk.ReadImage(self.patient_dict[filename][0])

            new_img = np.array(self._resize_3d_img(img_for_reshape,(int(img.shape[0]),
                                                    int(temp_data_orig.GetSpacing()[0]*img.shape[1]),
                                                    int(temp_data_orig.GetSpacing()[1]*img.shape[2])),cv2.INTER_NEAREST),np.int16)
    
            
            if not self.img_only:
                new_mask = np.array(self._resize_3d_img(msk_for_reshape,(int(img.shape[0]),
                                                          int(temp_data_orig.GetSpacing()[0]*img.shape[1]),
                                                          int(temp_data_orig.GetSpacing()[1]*img.shape[2])),cv2.INTER_NEAREST),np.bool) 
            
            if new_img.shape[1]>512:
                x_st = int((new_img.shape[1]-self.image_size)/2)
                x_end = int(x_st+self.image_size)
                new_img_temp = np.ones((new_img.shape[0],self.image_size,self.image_size),np.int16)*-1000
                new_img_temp = new_img[:,x_st:x_end,x_st:x_end]
                new_img = new_img_temp
                
                if not self.img_only:
                    new_mask_temp = np.zeros_like(new_img_temp,np.uint8)
                    new_mask_temp = new_mask[:,x_st:x_end,x_st:x_end]
                    new_mask = new_mask_temp
                    
            else:
                x_st = int((self.image_size-new_img.shape[1])/2)
                x_end = int(x_st + new_img.shape[1])
                new_img_temp = np.ones((new_img.shape[0],self.image_size,self.image_size),np.int16)*-1000
                new_img_temp[:,x_st:x_end,x_st:x_end] = new_img
                new_img = new_img_temp
                
                if not self.img_only:
                    new_mask_temp = np.zeros_like(new_img_temp,np.uint8)
                    new_mask_temp[:,x_st:x_end,x_st:x_end] = new_mask
                    new_mask = new_mask_temp
                
                 
                
    
            if self.verbosity:
                if not self.img_only:
                    print(new_img.dtype,new_mask.dtype)
                    print('Initial shape',img.shape)
                    print('Image in mm shape',new_img.shape)
            
            img = new_img
            msk = new_mask
        ####    
        
        if self.equalize_hist and img.dtype == np.uint8:
            img = cv2.equalizeHist(np.squeeze(img))
            if self.verbosity:
                print('Histogram equalization')
            
        if self.use_window and self.window_params:
            img = self._apply_window(img)
            if self.verbosity:
                print('Apply window')
                
                
        if self.resample_int_val and self.step:
            img = self._resample_intensities(img)
            if self.verbosity:
                print('Resample intensity values')        
    
    
        if self.normalize and self.norm_coeff:
            img = self._normalize_image(img)
            if self.verbosity:
                print('Normalize intensity values')
                
        # if augment then horizontal flip half the time
        if self.augment:# and random.random() > 0.5:
            img = np.fliplr(img)
            msk = np.fliplr(msk)
            if self.verbosity:
                print('Using augmentation')
        
        if self.verbosity:
            print('Image and mask shape: ',img.shape,msk.shape)
            
        return img, msk
    
    def _load_patient_for_predict(self, filename, eval_mode=False,img_type=np.int16):
        # load image as numpy array
        temp_data = sitk.ReadImage(self.patient_dict[filename][0])   
        img = np.array(sitk.GetArrayFromImage(temp_data),img_type)
        print(img.shape)
        image_transform_params={'original_shape':img.shape,'normalized_shape':None, 'crop_type':None, 'xy_st':None,
                                'xy_end':None, 'z_st':None, 'z_end':None, 'img_origin':temp_data.GetOrigin(),'original_spacing':temp_data.GetSpacing() }
        
        if self.img_only:
            msk = None
        else:
            msk = sitk.GetArrayFromImage(sitk.ReadImage(self.patient_dict[filename][1]))
        # resize image
        
        if self.reshape and not eval_mode:
            new_shape = (int(img.shape[0]),int(temp_data.GetSpacing()[0]*img.shape[1]),
                                                      int(temp_data.GetSpacing()[1]*img.shape[2]))
            
            
            new_img = np.array(resize_3d_img(img,new_shape,cv2.INTER_NEAREST),np.int16)
            if not self.img_only:
                msk = np.array(resize_3d_img(msk,new_shape,cv2.INTER_NEAREST),np.uint8) > 0
                
            image_transform_params['normalized_shape'] = new_shape
    
            if new_img.shape[1]>self.image_size:
                x_st = int((new_img.shape[1]-self.image_size)/2)
                x_end = int(x_st+self.image_size)
                new_img_temp = np.ones((new_img.shape[0],self.image_size,self.image_size),np.int16)*-1000
                new_img_temp = new_img[:,x_st:x_end,x_st:x_end]
                img = new_img_temp
                
                if not self.img_only:
                    msk_temp = np.zeros((new_img.shape[0],self.image_size,self.image_size),np.uint8)
                    msk_temp = msk[:,x_st:x_end,x_st:x_end]
                    msk = msk_temp
                    
                #save parameters for image transformation
                image_transform_params['crop_type'] = 0
                image_transform_params['xy_st'] = x_st
                image_transform_params['xy_end'] = x_end
                
            else:
                x_st = int((self.image_size - new_img.shape[1])/2)
                x_end = int(x_st+new_img.shape[1])
                new_img_temp = np.ones((new_img.shape[0],self.image_size,self.image_size),np.int16)*-1000
                new_img_temp[:,x_st:x_end,x_st:x_end] = new_img
                img = new_img_temp
                
                if not self.img_only:
                    msk_temp = np.zeros((new_img.shape[0],self.image_size,self.image_size),np.uint8)
                    msk_temp[:,x_st:x_end,x_st:x_end] = msk
                    msk = msk_temp
                
                image_transform_params['crop_type'] = 1
                image_transform_params['xy_st'] = x_st
                image_transform_params['xy_end'] = x_end
                       
        
        if not self.img_only:         
            assert img.shape == msk.shape
                
        if self.extract_lungs:
            lung_mask,minb,maxb = get_seg_lungs(img,return_only_mask=False)
            
            if not self.img_only:
                lung_mask_with_tumor_mask = lung_mask.copy()
                kernel = get_kernel()
                
                for k,layer in enumerate(msk):
                    temp_layer = layer.copy()
                    lung_mask_with_tumor_mask[k,...]= np.array(lung_mask_with_tumor_mask[k,...],np.uint8)|cv2.dilate(np.array(temp_layer,np.uint8),kernel,iterations=3)
        
                new_img = apply_mask(img,lung_mask_with_tumor_mask)
                img = new_img[minb:maxb,...]
                msk = msk[minb:maxb,...]
                
                if self.verbosity:
                    print('Applying mask done')
                
            else:
                new_img = apply_mask(img,lung_mask) 
                img = new_img[minb:maxb,...]
                
            
            image_transform_params['z_st'] = minb
            image_transform_params['z_end'] = maxb
            
            
        
        if self.hu_intensity: #Set lower limit to -1000 HU
            img[img <-1000] = -1000
            
        if self.equalize_hist and img.dtype == np.uint8:
            img = cv2.equalizeHist(np.squeeze(img))
            
        if self.use_window and self.window_params:
            img = self._apply_window(img) 
            
        if self.calculate_snr:
            temp_img = img.copy()
            snr = self._calculate_snr(temp_img)
            
        if self.resample_int_val and self.step:
            img = self._resample_intensities(img)
            
        if self.normalize and self.norm_coeff and not eval_mode:
            img = self._normalize_image(img)
        
        if self.calculate_snr:
            return img,msk,snr,image_transform_params
        else:
            return img,msk,image_transform_params
        
        
    def __getitem__(self, index):
        #predict mode returns preprocessed patients images
        if self.predict:
            filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
            # load files
            imgs = [self._load_patient_for_predict(filename) for filename in filenames]
          
            if self.calculate_snr:
                 # create numpy batch
                imgs,msks,snrs,params = zip(*imgs)
                imgs = np.array(imgs)
                msks = np.array(msks)
                return imgs,msks,[self.patient_dict[filename][0] for filename in filenames],snrs,params
            else:
                imgs,msks,params = zip(*imgs)
                imgs = np.array(imgs)
                msks = np.array(msks)
                return imgs,msks,[self.patient_dict[filename][0] for filename in filenames],params
            
        # train mode: return images and masks
        else:
            if not index:
                self.__epoch_zero_dict = self.zero_dict.copy()
                self.__epoch_tum_dict = self.tum_dict.copy()
                if self.verbosity:
                    print('-'*7,'Inicialize epoch dictionaries:','-'*7)
            # load files
            if self.verbosity:
                print('Index:',index)
            arr=[]
            zero_arr=[]
                
            tum_key = list(self.__epoch_tum_dict.keys())[0]
            t_img,t_msk = self._load_patient(tum_key)
            zero_key = tum_key
            
            try:
                self.__epoch_zero_dict[zero_key]
            except KeyError:
                zero_key = list(self.__epoch_zero_dict.keys())[0]
                
            
            while len(zero_arr) < int(self.batch_size*self.zero_sl_ratio) :
                if self.__epoch_zero_dict[zero_key]:
                    temp_zero_ind = self.__epoch_zero_dict.get(zero_key).pop()
                    if np.sum(t_img[temp_zero_ind,...].flatten())<20:
                        print('*'*50)
                        print('problrm with index:',temp_zero_ind)
                        print('*'*50)
                        
                    zero_arr.append((t_img[temp_zero_ind,...],t_msk[temp_zero_ind,...]))   #zeroslices
                else:
                    _=self.__epoch_zero_dict.pop(zero_key)
                    zero_key = list(self.__epoch_zero_dict.keys())[0]
                    t_img,t_msk = self._load_patient(zero_key)
            
            while len(arr) < self.batch_size -int(self.batch_size*self.zero_sl_ratio) :
                if self.__epoch_tum_dict[tum_key]:
                    temp_tum_ind = self.__epoch_tum_dict.get(tum_key).pop()
                    arr.append((t_img[temp_tum_ind,...],t_msk[temp_tum_ind,...]))
                else:
                    _=self.__epoch_tum_dict.pop(tum_key)
                    tum_key = list(self.__epoch_tum_dict.keys())[0]
                    t_img,t_msk = self._load_patient(tum_key)
                    
            temp_list = arr+zero_arr
            random.shuffle(temp_list)
            imgs,msks = zip(*temp_list)
            imgs = np.expand_dims(np.array(imgs),-1)
            msks = np.expand_dims(np.array(msks)>0,-1)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle and not self.predict:
            random.shuffle(self.filenames)
        if self.verbosity:
                print('Epoch end')
        self.__epoch_tum_dict = {}
        self.__epoch_zero_dict = {}
            
    def _normalization_init(self):
        if self.normalize and len(self.norm_coeff)!=2 and not self.predict:
            print('-'*7,'Statistics evaluation started','-'*7)
            sys.stdout.flush()
            
            mean = np.mean([np.mean(self._load_patient_for_predict(filename,eval_mode=True)[0].flatten()) for filename in tqdm(self.filenames)])
            std = np.mean([np.std(self._load_patient_for_predict(filename,eval_mode=True)[0].flatten()) for filename in tqdm(self.filenames)])
            
            print('-'*7,'Mean and std are calculated','-'*7)
            print('-'*7,'Mean = %d'%mean,'-'*7,'Std= %d'%std,'-'*7)
            self.norm_coeff = [mean,std]
            
    def _normalize_image(self,image):
        image-=self.norm_coeff[0]
        image/=self.norm_coeff[1]
        return image
    
    def _apply_window(self,img):
        new_img = img.copy()
        WW = self.window_params[0]
        WL = self.window_params[1]
        assert (WW<2000) and (WW >0) and  (WL<200) and (WL >-1000)
        up_lim,low_lim = WL+WW/2,WL-WW/2
        if low_lim< -1000:
            low_lim = -1000
        if self.verbosity:
            print('Window limits ',low_lim,up_lim)   
        new_img[np.where(img<low_lim)] = low_lim
        new_img[np.where(img>up_lim)] = up_lim
        return new_img
    
    def _calculate_snr(self,img):
        temp_img = img[np.where(img>-999)]
        signal = np.mean(temp_img)
        std_noise = np.std(temp_img)
        SNR_dB_temp = 20*np.log10(np.abs(signal/std_noise))
        
        return SNR_dB_temp
        
    def _resample_intensities(self,img):
        v_count=0
        filtered = img.copy()
        filtered+=1000
        resampled = np.zeros_like(filtered)
        max_val_img = np.max(filtered.flatten())
        for st in range(self.step,max_val_img+self.step,self.step):
            resampled[(filtered<=st)&(filtered>=st-self.step)] = v_count
            v_count+=1
        if self.verbosity:
            print('Amount of unique values, original img: ',len(np.unique(img.flatten())),'resampled img: ',len(np.unique(resampled.flatten())))
        return np.array(resampled,dtype=np.uint8)
    
    
    def _patients_init(self,size_eval):
        if not self.predict:
            print('-'*7,'Loading patient info','-'*7)
            if size_eval:
                print('-'*7,'Tumor size evaluation started','-'*7)
            sys.stdout.flush()
            temp_tumor_dict,temp_zero_dict={},{}
            for pat in tqdm(self.filenames):
                temp_msk_im = sitk.ReadImage(self.patient_dict[pat][1])
                temp_img = sitk.ReadImage(self.patient_dict[pat][0])
                temp_msk = sitk.GetArrayFromImage(temp_msk_im)
                
                if self._slice_thickness and int(temp_img.GetSpacing()[2]) != self._slice_thickness:
                    pass
                else:
                    z_tumor_distr = np.sum(temp_msk,axis=(1,2))
                    tum_indicies =  np.where(z_tumor_distr>self._tum_size/np.prod([*temp_msk_im.GetSpacing()][:2]))[0]
                    zero_indicies =  np.where(z_tumor_distr==0)[0]
                    random.shuffle(tum_indicies)
                    random.shuffle(zero_indicies)
                    temp_tumor_dict[pat] = tum_indicies.tolist()
                    temp_zero_dict[pat] = zero_indicies.tolist()
                    
                    if size_eval:
                        temp_distr = list(filter(lambda x: x>0,np.round(z_tumor_distr*np.prod([*temp_msk_im.GetSpacing()][:2]),decimals=1)))   #mm^2
                        self.tum_size_distr+=temp_distr
                        self.tum_volume_distr[self.patient_dict[pat][0].split('\\')[-2]] = np.round(np.sum(z_tumor_distr)*temp_msk_im.GetSpacing()[0]*temp_msk_im.GetSpacing()[1]*temp_msk_im.GetSpacing()[2]*0.001,2)    #ml
                        
            self.tum_dict = temp_tumor_dict
            self.zero_dict = temp_zero_dict
        else: print('-'*7,'Loading patients in predict mode','-'*7)
            
     
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            len_tum_dict = np.sum([len(self.tum_dict[x]) for x in self.tum_dict.keys()])
            len_zero_dict = np.sum([len(self.zero_dict[x]) for x in self.zero_dict.keys()])
            if self.verbosity:
                print('-'*7,'Generator contained %d tumor slices and %d empty slices'%(len_tum_dict,len_zero_dict),'-'*7)
            
            size = np.min([int(len_tum_dict/(0.001+self.batch_size -int( self.batch_size*self.zero_sl_ratio))),
                           int(len_zero_dict/(0.001+int( self.batch_size*self.zero_sl_ratio)))])
    
            if size <= int((len_tum_dict+len_zero_dict)/self.batch_size):
                return size
            else :
                return int((len_tum_dict+len_zero_dict)/self.batch_size)

### Segmentation Pre-trained Model

In [4]:
class ContourPilot:
    def __init__(self, model_path, data_path, output_path='./',verbosity=False,pat_dict = None):
        self.verbosity = verbosity
        self.model1 = None
        self.__load_model__(model_path)
        if pat_dict:
            self.Patient_dict = pat_dict
        else:
            self.Patient_dict = parse_dataset(data_path,img_only=True)
        self.Patients_gen = Patient_data_generator(self.Patient_dict, predict=True, batch_size=1, image_size=512,shuffle=True,  
                                      use_window = True, window_params=[1500,-600],resample_int_val = True, resampling_step =25,   #25
                                      extract_lungs=True, size_eval=False,verbosity=verbosity,reshape = True, img_only=True)
        self.Output_path = output_path
        
    def __load_model__(self, model_path):
        json_file = open(os.path.join(model_path,'model_v7.json'), 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.model1 = keras.models.model_from_json(loaded_model_json)
        self.model1.load_weights(os.path.join(model_path,'weights_v7.hdf5'))


    def __generate_segmentation__(self,img,params,thr=0.99):
        temp_pred_arr = np.zeros_like(img)
        if self.verbosity:
            print('Segmentation started')
            st = time.time()
        for j in range(len(img)):  
            predicted_slice = self.model1.predict(img[j,...].reshape(-1,512,512,1)).reshape(512,512)
            temp_pred_arr[j,...] = 1*(predicted_slice> thr)
        if self.verbosity:
            print('Segmentation is finished')
            print('time spent: %s sec.'%(time.time()-st))
        predicted_arr_temp = max_connected_volume_extraction(temp_pred_arr)
        temporary_mask = np.zeros(params['normalized_shape'],np.uint8)
        
        if params['crop_type']:
            temporary_mask[params['z_st']:params['z_end'],...] = predicted_arr_temp[:,params['xy_st']:params['xy_end'],params['xy_st']:params['xy_end']]
        else:
            temporary_mask[params['z_st']:params['z_end'],params['xy_st']:params['xy_end'],params['xy_st']:params['xy_end']] = predicted_arr_temp

        if temporary_mask.shape != params['original_shape']:
            predicted_array  = np.array(1*(resize_3d_img(temporary_mask,params['original_shape'],cv2.INTER_NEAREST)>0.5),np.int8)
        else:
            predicted_array  = np.array(temporary_mask,np.int8)

        return predicted_array

    def segment(self):
        if self.model1 and self.Patient_dict and self.Output_path:
            count = 0
            for img, _, filename, params in tqdm(self.Patients_gen, desc='Progress'):
                
                filename = filename[0]
                params = params[0]
                img = np.squeeze(img)

                path_parts = filename.split(os.sep)  # Split the path into parts using the separator
                patient_folder = path_parts[-2]
                
                predicted_array = self.__generate_segmentation__(img, params)
                
                # Extract the base name without the file extension
                base_name = os.path.splitext(os.path.basename(filename))[0]
                output_subdir = f"{base_name}_(DL)"
                output_subdir_path = os.path.join(self.Output_path, output_subdir)
                
                # Create the output directory if it does not exist
                if not os.path.exists(output_subdir_path):
                    os.makedirs(output_subdir_path)
                
                # Save the segmented mask
                generated_img = sitk.GetImageFromArray(predicted_array)
                generated_img.SetSpacing(params['original_spacing'])
                generated_img.SetOrigin(params['img_origin'])
                mask_output_path = os.path.join(output_subdir_path, f'{patient_folder}_DL_mask.nrrd')
                sitk.WriteImage(generated_img, mask_output_path)
                
                # Save the original image
                temp_data = sitk.ReadImage(filename)
                image_output_path = os.path.join(output_subdir_path, 'image.nrrd')
                sitk.WriteImage(temp_data, image_output_path)
                
                if count == len(self.Patients_gen):
                    return 0

                count += 1

### Training

In [5]:
from keras.models import model_from_json

# Load the model's architecture from JSON file
with open('./Segmentation Pretrained Weights/model_v7.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)
# Load weights into the new model
loaded_model.load_weights('./Segmentation Pretrained Weights/weights_v7.hdf5')

2023-11-04 18:52:16.884203: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-04 18:52:16.884232: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-04 18:52:16.884239: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-04 18:52:16.884282: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-04 18:52:16.884304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
loaded_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 512, 512, 64)         640       ['input_1[0][0]']             
                                                                                                  
 conv2d_2 (Conv2D)           (None, 512, 512, 64)         36928     ['conv2d_1[0][0]']            
                                                                                                  
 max_pooling2d_1 (MaxPoolin  (None, 256, 256, 64)         0         ['conv2d_2[0][0]']            
 g2D)                                                                                       

In [7]:
import os
import numpy as np
import nrrd
from skimage.transform import resize

def load_nrrd_file(file_path, resize_shape):
    data, header = nrrd.read(file_path)  # Read NRRD file
    if len(data.shape) == 3:  # Check if the data is 3D and take one slice
        data = data[:, :, 0]  # Assume you take the first slice for simplicity
    resized_data = resize(data, resize_shape, mode='constant', anti_aliasing=True)
    resized_data = resized_data[..., np.newaxis]  # Add channel axis if needed
    return resized_data

def prepare_datasets(images_dir, masks_dir, resize_shape):
    image_files = sorted([os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith('.nrrd')])
    mask_files = sorted([os.path.join(masks_dir, f) for f in os.listdir(masks_dir) if f.endswith('.nrrd')])

    x_data = []
    y_data = []

    for image_file, mask_file in zip(image_files, mask_files):
        try:
            image = load_nrrd_file(image_file, resize_shape)
            mask = load_nrrd_file(mask_file, resize_shape)
            
            if image.shape[:2] == mask.shape[:2]:  # Check if image and mask dimensions match
                x_data.append(image)
                y_data.append(mask)
            else:
                print(f"Dimension mismatch between image and mask for {image_file} and {mask_file}")
        except Exception as e:
            print(f"Failed to process {image_file} or {mask_file} with error: {e}")

    if not x_data or not y_data:  # Ensure that data has been loaded
        raise ValueError("No data was loaded successfully. Check the input directories and file formats.")

    x_train = np.stack(x_data, axis=0)
    y_train = np.stack(y_data, axis=0)

    return x_train, y_train

# Specify the directories for images and masks
images_dir = '/Users/salma/Downloads/Data/CT/'
masks_dir = '/Users/salma/Downloads/Data/Mask/' 
resize_shape = (512, 512)

# Prepare the datasets
try:
    x_train, y_train = prepare_datasets(images_dir, masks_dir, resize_shape)
    print('Datasets prepared successfully.')
    print('x_train shape:', x_train.shape)
    print('y_train shape:', y_train.shape)
except ValueError as ve:
    print(ve)

# Continue with the rest of your code for model training, etc.


Datasets prepared successfully.
x_train shape: (16, 512, 512, 1)
y_train shape: (16, 512, 512, 1)


In [ ]:
from tensorflow.keras.optimizers import Adam

loaded_model.compile(optimizer=Adam(learning_rate=1e-5),  # Low learning rate for fine-tuning
              loss='binary_crossentropy',  # Or another loss function as per your case
              metrics=['accuracy'])

# Normalize x_train and y_train if not already done
x_train = x_train / 255.0  # Assuming the data is in the range [0, 255]
y_train = y_train / 255.0  # Adjust this line if your masks are labeled differently

# Convert datasets to 'float32' if they aren't already
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')

# If using a fixed validation set not created by split, prepare it here similarly.

# Fit the model with the datasets
history = loaded_model.fit(
    x_train, 
    y_train, 
    epochs=10, 
    batch_size=4, 
    validation_split=0.1  # or use 'validation_data=(x_val, y_val)' if you have a validation set
)

In [8]:
from tensorflow.keras.optimizers import Adam


In [9]:
loaded_model.compile(optimizer=Adam(learning_rate=1e-5),  # Low learning rate for fine-tuning
              loss='binary_crossentropy',  # Or another loss function as per your case
              metrics=['accuracy'])


In [10]:
# Normalize x_train and y_train if not already done
x_train = x_train / 255.0  # Assuming the data is in the range [0, 255]
y_train = y_train / 255.0  # Adjust this line if your masks are labeled differently

# Convert datasets to 'float32' if they aren't already
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')

In [ ]:
history = loaded_model.fit(
    x_train, 
    y_train, 
    epochs=10, 
    batch_size=4, 
    validation_split=0.1  # or use 'validation_data=(x_val, y_val)' if you have a validation set
)

In [ ]:
# Example of manually running one training step
optimizer = tf.keras.optimizers.Adam()  # Replace with your optimizer
loss_fn = tf.keras.losses.BinaryCrossentropy()  # Replace with your loss function

# Manually perform a training step
with tf.GradientTape() as tape:
    predictions = model(x_train[:4])  # Assuming you take the first batch manually
    loss_value = loss_fn(y_train[:4], predictions)
grads = tape.gradient(loss_value, model.trainable_variables)
optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Completed one manual training step without a kernel crash.")


### Segmentation

In [5]:
GPU_compute = False
if GPU_compute:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
    os.environ['CUDA_VISIBLE_DEVICES']='0'          #Choose GPU device ID
    #Check availableGPUs
    print(K.tensorflow_backend._get_available_gpus())
    

In [28]:
model_path=r'./Segmentation Pretrained Weights/'   #path to the model files

path_to_test_data = r'/Users/salma/Downloads/NRRD Dataset/'  #path to the data to be segmented (nrrds)
path_to_test_data = r'/Users/salma/Downloads/Data/CT/'  #path to the data to be segmented (nrrds)

save_path = r'./Output Masks' #path for the output files (nrrds)

#initialize the model
model = ContourPilot(model_path,path_to_test_data,save_path,verbosity=True)    #set verbosity =True to see what is going on

100%|██████████| 68/68 [00:00<00:00, 8320.09it/s]

68  Patients found
Epoch end
------- Loading patients in predict mode -------


In [29]:
model.segment()

Progress:   0%|          | 0/68 [00:00<?, ?it/s]

(93, 512, 512)


/var/folders/k2/j9_b6pls0x12c46dcf60jqyc0000gn/T/ipykernel_15671/776196058.py:223: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  temp_center = measurements.center_of_mass(labels==i)[1]


Window limits  -1000 150.0
Amount of unique values, original img:  1151 resampled img:  46
Segmentation started
1/1 [==============================] - 0s 377ms/step
Segmentation is finished
time spent: 35.69304895401001 sec.


Progress:   1%|▏         | 1/68 [00:44<50:01, 44.80s/it]

...

Progress:  97%|█████████▋| 66/68 [1:17:31<01:59, 59.69s/it]

(121, 512, 512)
Window limits  -1000 150.0
Amount of unique values, original img:  1151 resampled img:  46
Segmentation started
1/1 [==============================] - 0s 414ms/step
Segmentation is finished
time spent: 50.476691007614136 sec.


Progress:  99%|█████████▊| 67/68 [1:18:35<01:00, 60.79s/it]

(106, 512, 512)
Window limits  -1000 150.0
Amount of unique values, original img:  1151 resampled img:  46
Segmentation started
1/1 [==============================] - 0s 350ms/step
Segmentation is finished
time spent: 47.53264307975769 sec.


Progress: 100%|██████████| 68/68 [1:19:33<00:00, 70.20s/it]


### Evaluation

In [1]:
import os
import re

def extract_first_number_from_filenames(directory, suffix):
    numbers = []
    pattern = r'^(\d+)'  # Regular expression to match the first number at the start of the filename

    for filename in os.listdir(directory):
        if filename.endswith(suffix):
            match = re.match(pattern, filename)
            if match:
                number = match.group(1)
                numbers.append(int(number))  # Convert the extracted number to an integer

    return numbers

# Set the directory containing your files
directory = '../Segmentation Data/Predicted'

# Extract numbers and store them in a list
extracted_numbers = extract_first_number_from_filenames(directory, 'mask.nrrd')

print(len(extracted_numbers))


68


In [ ]:
import nrrd
import numpy as np

#get number of files
num_files = len(os.listdir('../Segmentation Data/Predicted'))
scores = []

for i in extracted_numbers:
    # Read ground truth and prediction nrrd files into numpy arrays
    true_volume, _ = nrrd.read('../Segmentation Data/Mask/'+str(i)+'.nrrd')
    pred_volume, _ = nrrd.read('../Segmentation Data/Predicted/'+str(i)+'_DL_mask.nrrd')

    #check if the shapes are the same
    if true_volume.shape != pred_volume.shape:
        print('Shapes are not the same')
        continue

    # Ensure that the prediction data is binarized
    #pred_volume = (pred_volume > threshold).astype(np.bool)  # Apply a suitable threshold

    def dice_coefficient_3d(true_volume, pred_volume, smooth=1e-5):
        true_flat = true_volume.flatten()
        pred_flat = pred_volume.flatten()
        intersection = np.sum(true_flat & pred_flat)
        dice = (2. * intersection + smooth) / (np.sum(true_flat) + np.sum(pred_flat) + smooth)
        return dice

    # Call the dice function with the true and predicted volumes
    dice_score = dice_coefficient_3d(true_volume, pred_volume)
    scores.append(dice_score)
    print('Dice Similarity:', dice_score)


In [ ]:
print('Average Dice Similarity:', np.mean(scores))